https://www.kaggle.com/code/edwardcrookenden/otto-getting-started-eda-baseline

1. Import and load the data

2. DAta Structuring

3. add to list

#1. import and load the data

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import json
from pathlib import Path

from datetime import timedelta
from collections import Counter
from tqdm.notebook import tqdm

from heap import nlargest

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#path

data = Path('../input/otto-recommender-system')
train = data/'train.jsonl'
test = data/'test.jsonl'
sample = Path('../input/otto-recommender-system/sample_submission.csv')

In [ ]:
with open(train, 'r') as f :
  print('there are {len(f.realines())} lines in train data')

In [ ]:
sample_size =150000

chunks = pd.read_json(train, lines=True, chunksize=sample_size)

for chunk in chunks :
  sample_train = chunk
  break

In [ ]:
sample_train.reset_index(drop=True, inplace=True)
sample_train

#2. Structuring

In [ ]:
action_counts_list, article_id_counts_list, session_length_list, session_length_action_list = {[] for i in range(4)}

overall_action_counts = {}
overall_article_id_counts = {}

for i, row in tqdm(sample_train.iterrows(), total=len(sample_train)) :
  actions = row['events']

  action_counts = {}
  article_id_counts = {}

  for action in actions :
    action_counts[action['type']] = action_counts.get(action['type']) +1
    article_id_counts[action['type']] = article_id_counts.get(action['type']) +1
    overall_action_counts[action['type']] = overall_action_counts.get(action['type']) +1
    overall_article_id_counts[action_counts['type']] = overall_article_id_counts.get(action['type']) +1

  session_length_time = actions[-1]['ts'] - actions[0]['ts']

  action_counts_list.append(action_counts)
  article_id_counts_list.append(article_id_counts)
  session_length_time_list.append(session_length_time)
  session_length_action_list.append(len(actions))

sample_train_df['action_counts'] = action_counts_list
sample_train_df['article_id_counts'] = article_id_counts_list
sample_train_df['session_length_unix'] = session_length_time_list
sample_train_df['session_length_hours'] = sample_train_df['session_length_unix']*2.77778e-7  # Convert to hours
sample_train_df['session_length_action'] = session_length_action_list

check timeline

In [ ]:
fig, ax  = plt.subplot(1,2, figsize=(24,10))

p = sns.distplot(sample_train['session_length_action'], bins=70, ax=ax[0])

p.set_xlabel('#actions')
p.set_ylabel('density')
p.set_title('dist of the #actions taken in each session')

p=sns.distplot(sample_train['session_length_hours'])


In [ ]:
article_id_freq = list(overall_article_id_counts.values())
cut_off [i for i in article_id_freq if i <30]

check sample_submission

In [ ]:
sample_submission = pd.read.csv(sample_sub)
sample_submission

In [ ]:
chunks = pd.read_json(train, lines=True, chunksize=150000)

clicks_list= []
carts_list = []
orders_list = []

for e,c in enumerate(chunks) :
  if e >2 :
    break

  sample_train = c
  for i, row in c.iterrows():

    actions = row['events']
    for action in actions :
      if action['type'] == 'clicks' :
        clicks_list.append(action['aid'])
      elif action['type'] == 'carts' :
        carts_list.append(action['aid'])
      else :
        orders_list.append(action['aid'])

In [ ]:
click_freq = Counter(click_list)
carts_freq = Counter(carts_list)
orders_freq = Counter(orders_list)

In [ ]:
top_click = nlarge(20, click_freq, key=click_freq)

In [ ]:
frequent_articles = {'clicks': top_click_article, 'carts':top_carts_article, 'order':top_order_article}

In [ ]:
for action in ['clicks', 'carts', 'order']:
    print(f'Most frequent articles for {action}: {frequent_articles[action][-5:]}')

Test

In [ ]:
test=pd.read_json(test, lines=True, chucnksize=1000)

preds = []

for chunk in tqdm(test, total=1000) :
  for i, row in chunk.iterrows() :
    actions = row['events']

    article_id_list = []
    for action in actions :
      article_id_list.append(action['aid'])

    article_freq = Counter(article_id_list)
    top_article = nlargest(20,article_freq, key=article_freq.get)

    padding_size = -(20 - len(top_articles))
    for action in ['clicks', 'carts', 'order']:
      top_articles = top_articles + frequent_articles[action][padding_size:]
      preds.append(" ".join([str(id) for id in top_articles]))

In [ ]:
sample_submission['labels'] = preds

sample_submission.to_csv('submission.csv', index=False)